In [23]:
import sys
import numpy as np
from keras.layers import Input, Dense, Lambda, Layer
from keras.initializers import Constant
from keras.models import Model
from keras import backend as K
import preprocessing
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
from keras import layers, models

In [2]:
# Custom loss layer
class CustomMultiLossLayer(Layer):
    def __init__(self, nb_outputs=2, **kwargs):
        self.nb_outputs = nb_outputs
        self.is_placeholder = True
        super(CustomMultiLossLayer, self).__init__(**kwargs)
        
    def build(self, input_shape=None):
        # initialise log_vars
        self.log_vars = []
        for i in range(self.nb_outputs):
            self.log_vars += [self.add_weight(name='log_var' + str(i), shape=(1,),
                                              initializer=Constant(0.), trainable=True)]
        super(CustomMultiLossLayer, self).build(input_shape)

    def multi_loss(self, ys_true, ys_pred):
        assert len(ys_true) == self.nb_outputs and len(ys_pred) == self.nb_outputs
        loss = 0
        for y_true, y_pred, log_var in zip(ys_true, ys_pred, self.log_vars):
            precision = K.exp(-log_var[0])
            loss += K.sum(precision * (y_true - y_pred)**2. + log_var[0], -1)
        return K.mean(loss)

    def call(self, inputs):
        ys_true = inputs[:self.nb_outputs]
        ys_pred = inputs[self.nb_outputs:]
        loss = self.multi_loss(ys_true, ys_pred)
        self.add_loss(loss, inputs=inputs)
        # We won't actually use the output.
        return K.concatenate(inputs, -1)

In [16]:
N = 100
nb_epoch = 3
batch_size = 20
Q = 1
D1 = 1  # first output
D2 = 1  # second output

In [41]:
input_shape = (128,128,3)
def get_prediction_model():
    inp = Input(shape=input_shape)
    conv1 = Conv2D(64, (3, 3), activation='relu')(inp)
    conv2 = Conv2D(64, (3, 3), activation='relu')(conv1)
    max_pool = MaxPooling2D((2, 2))(conv2)
    flatten = Flatten()(max_pool)
    dense_shared = Dense(128, activation='relu')(flatten)
    classification_output = layers.Dense(1, activation='sigmoid', name='classification')(dense_shared)
    regression_output = layers.Dense(1, activation='linear', name='regression')(dense_shared)
    return Model(inp, [regression_output, classification_output])

def get_trainable_model(prediction_model):
    inp = Input(shape=(128,128,3), name='inp')
    regression_pred, classification_pred = prediction_model(inp)
    regression_true = Input(shape=(D1,), name='regression_true')
    classification_true = Input(shape=(D2,), name='classification_true')
    out = CustomMultiLossLayer(nb_outputs=2)([regression_true, classification_true, regression_pred, classification_pred])
    return Model([inp, regression_true, classification_true], out)

prediction_model = get_prediction_model()
trainable_model = get_trainable_model(prediction_model)
trainable_model.compile(optimizer='adam', loss=None)
assert len(trainable_model.layers[-1].trainable_weights) == 2  # two log_vars, one for each output
assert len(trainable_model.losses) == 1
prediction_model.summary()
trainable_model.summary()

Model: "model_26"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_17 (Conv2D)             (None, 126, 126, 64  1792        ['input_10[0][0]']               
                                )                                                                 
                                                                                                  
 conv2d_18 (Conv2D)             (None, 124, 124, 64  36928       ['conv2d_17[0][0]']              
                                )                                                          

In [7]:
train_gen, val_gen = preprocessing.preprocess()

Found 8 validated image filenames.
Found 1 validated image filenames.


In [37]:
train_gen.next()

(array([[[[0.69945747, 0.62886924, 0.62886924],
          [0.6994332 , 0.628845  , 0.628845  ],
          [0.6994089 , 0.62882066, 0.62882066],
          ...,
          [0.45813605, 0.37970465, 0.3365674 ],
          [0.4581846 , 0.37975323, 0.33661598],
          [0.45823315, 0.37980178, 0.33666453]],
 
         [[0.69411767, 0.62352943, 0.62352943],
          [0.69411767, 0.62352943, 0.62352943],
          [0.69411767, 0.62352943, 0.62352943],
          ...,
          [0.46478993, 0.38635856, 0.34322128],
          [0.46480206, 0.3863707 , 0.34323344],
          [0.46481422, 0.38638285, 0.3432456 ]],
 
         [[0.69411767, 0.62352943, 0.62352943],
          [0.69411767, 0.62352943, 0.62352943],
          [0.69411767, 0.62352943, 0.62352943],
          ...,
          [0.4719479 , 0.39351654, 0.3503793 ],
          [0.47199646, 0.3935651 , 0.35042784],
          [0.47204506, 0.3936137 , 0.3504764 ]],
 
         ...,
 
         [[0.7058824 , 0.63529414, 0.63529414],
          [0.70588

In [42]:
hist = trainable_model.fit(train_gen, epochs=nb_epoch, batch_size=batch_size, verbose=0)

ValueError: in user code:

    File "C:\Users\sophi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\sophi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\sophi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\sophi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1050, in train_